<a href="https://colab.research.google.com/github/RubenCarty/Master-Artificial-Intelligence-UNIR/blob/main/NLP%20/%20mia07_lab_red_transformers_RubenQuispe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

# **Actividad2 :Laboratorio: word embedings y transformers para clasificación de texto**

   # **Curso: Natural Language Proccessing**

# **Máster Universitario en Inteligencia Artificial**
# [Student: RUBEN QUISPE](https://www.linkedin.com/in/ruben-quispe-l/)

</center>

# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python".

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

# Librerías

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import pathlib
import spacy
import en_core_web_sm

Se importan las bibliotecas necesarias para el manejo de arrays (numpy), para trabajar con TensorFlow y Keras en la construcción de modelos de deep learning (tensorflow y keras), y para manipular archivos y directorios (os, pathlib). También se importa spacy y en_core_web_sm para la manipulación de textos.

# Descarga de Datos

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 16s 1us/step


Se descarga el dataset "20 Newsgroups" de una URL específica y se descomprime en la ubicación predeterminada.

In [3]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['rec.sport.hockey', 'talk.politics.mideast', 'talk.politics.misc', 'sci.space', 'rec.motorcycles', 'rec.sport.baseball', 'rec.autos', 'sci.electronics', 'comp.sys.mac.hardware', 'talk.religion.misc', 'sci.med', 'misc.forsale', 'comp.graphics', 'alt.atheism', 'comp.sys.ibm.pc.hardware', 'soc.religion.christian', 'comp.os.ms-windows.misc', 'sci.crypt', 'comp.windows.x', 'talk.politics.guns']


Se obtienen los nombres de los directorios que corresponden a las diferentes categorías del dataset y se imprimen.

## Directorio del Dataset

In [4]:
print(data_dir)

/root/.keras/datasets/20_newsgroup


Se imprime el directorio donde se descomprimió el dataset

In [5]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['38712', '38443', '38572', '39646', '38817']


Se listan los nombres de archivos dentro de la categoría comp.graphics y se imprimen los primeros 5.

In [6]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "37261").read())

Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:37261 alt.graphics:519 comp.graphics.animation:2614
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!zaphod.mps.ohio-state.edu!darwin.sura.net!dtix.dt.navy.mil!oasys!lipman
From: lipman@oasys.dt.navy.mil (Robert Lipman)
Newsgroups: comp.graphics,alt.graphics,comp.graphics.animation
Subject: CALL FOR PRESENTATIONS: Navy SciViz/VR Seminar
Message-ID: <32850@oasys.dt.navy.mil>
Date: 19 Mar 93 20:10:23 GMT
Article-I.D.: oasys.32850
Expires: 30 Apr 93 04:00:00 GMT
Reply-To: lipman@oasys.dt.navy.mil (Robert Lipman)
Followup-To: comp.graphics
Distribution: usa
Organization: Carderock Division, NSWC, Bethesda, MD
Lines: 65


			CALL FOR PRESENTATIONS
	
      NAVY SCIENTIFIC VISUALIZATION AND VIRTUAL REALITY SEMINAR

			Tuesday, June 22, 1993

	    Carderock Division, Naval Surface Warfare Center
	      (formerly the David Taylor Research Center)
			  Bethesda, Maryland

SPONSOR: NESS (Navy Engineering Software System) is sponsori

Se lee y se imprime el contenido de un archivo de ejemplo en la categoría comp.graphics.

### Archivos de Otra Categoría

In [7]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['179111', '178944', '178697', '179072', '178742']


Se listan los nombres de archivos dentro de la categoría talk.politics.misc y se imprimen los primeros 5.

In [8]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

Se lee y se imprime el contenido de un archivo de ejemplo en la categoría talk.politics.misc.

In [9]:
list_all_dir = [
    'alt.atheism',
    'comp.graphics',
    'comp.sys.mac.hardware',
    'comp.windows.x',
    'misc.forsale',
    'rec.autos',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'sci.crypt',
    'sci.med',
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
    'talk.politics.misc',
    'talk.religion.misc'
]

Se define una lista de categorías a utilizar en el análisis.

### Procesamiento de Archivos

In [10]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in list_all_dir:
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.misc', 'talk.religion.misc']
Number of samples: 14997


 Se procesan los archivos de cada categoría, leyendo el contenido y almacenándolo en listas de samples y labels. Cada categoría recibe un índice único.

# Mezclando los datos para separarlos en Traning y Test

In [11]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)
keras.utils.set_random_seed(seed)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

Explicación:

* Semilla (seed): Establece una semilla para asegurar que los resultados sean reproducibles.
* Mezcla (shuffle): Mezcla los samples (muestras) y labels (etiquetas) para asegurar que los datos de entrenamiento y prueba sean representativos de la variedad de datos disponibles.
* División en Entrenamiento y Validación: Se divide el conjunto de datos en entrenamiento (80%) y validación (20%).
- Reflexión: La mezcla de los datos antes de dividirlos garantiza que ambas particiones (entrenamiento y validación) sean representativas, evitando sesgos que puedan afectar el rendimiento del modelo.

# Tokenización de las palabras con TextVectorization

In [12]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

* TextVectorization: Crea un vectorizador de texto que limita el vocabulario a 20,000 palabras y establece una longitud máxima de secuencia de 200.
* Adaptación del Vectorizador: Se adapta el vectorizador al conjunto de datos de entrenamiento, permitiéndole aprender el vocabulario.
- Reflexión: La tokenización es crucial para convertir el texto en una representación numérica que el modelo pueda procesar. El ajuste del vectorizador al dataset de entrenamiento asegura que el vocabulario relevante sea capturado.

# Visualización y Verificación de la Tokenización

In [13]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

Primeros 5 tokens del vocabulario, donde [UNK] representa palabras desconocidas.

In [14]:
len(vectorizer.get_vocabulary())

20000

20000: Longitud del vocabulario.

# Viendo la salida de Vectorizer

In [15]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 4079, 1984,   17,    2, 5135])

array([ 2, 4079, 1984, 17, 2, 5135]): Tokens correspondientes a la frase "the cat sat on the mat".

In [16]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 4079, 1984,   17,    2, 5135,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

Reflexión: Verificar el vocabulario y las secuencias tokenizadas asegura que el proceso de tokenización está funcionando correctamente, proporcionando una buena base para el entrenamiento del modelo.

In [17]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [18]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 4079, 1984, 17, 2, 5135]

# Tokenización de los datos de entrenamiento y validación

In [19]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

Explicación:

- Tokenización: Aplica el vectorizador a las muestras de entrenamiento y validación, convirtiéndolas en secuencias numéricas.
- Conversión a NumPy: Convierte las secuencias y etiquetas a arrays de NumPy.
- Reflexión: Este paso transforma el texto en un formato adecuado para ser procesado por redes neuronales, asegurando una correcta entrada de datos al modelo.

# Creación y entrenamiento del modelo. Transformers

In [20]:
from tensorflow.keras import layers
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [21]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

Explicación:

- TransformerBlock: Implementa un bloque de transformador con atención multi-cabeza y una red neuronal feed-forward.
- TokenAndPositionEmbedding: Combina embeddings de tokens y posiciones, cruciales para que el transformador entienda la posición de las palabras en la secuencia.

### Construcción del Modelo:

In [22]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
num_tokens = len(voc) + 2

maxlen = 200
vocab_size = num_tokens

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

modeloTransformers = keras.Model(inputs=inputs, outputs=outputs)

Explicación:

- Estructura del Modelo: Construye el modelo de transformador, agregando capas de embedding, bloques de transformador, pooling, y capas densas para la clasificación final.

## Entrenamiento del Modelo:

In [23]:
modeloTransformers.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloTransformers.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloTransformers.summary())

Epoch 1/20
94/94 [==============================] - 77s 776ms/step - loss: 2.6455 - acc: 0.1109 - val_loss: 2.5149 - val_acc: 0.1777
Epoch 2/20
94/94 [==============================] - 72s 770ms/step - loss: 2.3198 - acc: 0.2316 - val_loss: 1.9955 - val_acc: 0.3968
Epoch 3/20
94/94 [==============================] - 77s 820ms/step - loss: 1.8106 - acc: 0.4104 - val_loss: 1.4374 - val_acc: 0.5505
Epoch 4/20
94/94 [==============================] - 64s 686ms/step - loss: 1.3341 - acc: 0.5578 - val_loss: 1.0937 - val_acc: 0.6442
Epoch 5/20
94/94 [==============================] - 64s 685ms/step - loss: 1.0036 - acc: 0.6679 - val_loss: 1.1865 - val_acc: 0.6119
Epoch 6/20
94/94 [==============================] - 69s 740ms/step - loss: 0.8298 - acc: 0.7294 - val_loss: 1.0961 - val_acc: 0.6445
Epoch 7/20
94/94 [==============================] - 68s 725ms/step - loss: 0.6774 - acc: 0.7805 - val_loss: 1.0161 - val_acc: 0.6786
Epoch 8/20
94/94 [==============================] - 68s 726ms/step - 

Resultados de Entrenamiento:

- El modelo muestra un aumento en la precisión de entrenamiento y validación a lo largo de las épocas, alcanzando una precisión de validación de alrededor del 82.6%.
- Reflexión: Utilizar un modelo de transformador permite capturar relaciones complejas en el texto, mejorando la capacidad del modelo para clasificar textos en diferentes categorías.

# Evaluación

In [24]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloTransformers(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 1s 1s/step


'comp.graphics'

In [25]:
probabilities = end_to_end_model.predict(
    [["politics and federal courts law that people understand with politician and elects congressman"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 42ms/step


'talk.politics.guns'

In [26]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 37ms/step


'alt.atheism'

Resultados:

- "comp.graphics": Correctamente clasifica un mensaje sobre gráficos computacionales.
- "talk.politics.guns": Correctamente clasifica un mensaje sobre política y leyes federales.
- "alt.atheism": Correctamente clasifica un mensaje sobre religión.
- Reflexión: Evaluar el modelo con ejemplos reales asegura que el modelo no solo funciona bien en los datos de entrenamiento y validación, sino que también generaliza bien a nuevos datos.

Para mejorar la precisión del modelo en la clasificación de texto usando embeddings y transformadores, podemos considerar varias técnicas. Aquí hay algunas estrategias comunes para optimizar un modelo de NLP:

1. Aumento del tamaño del modelo: Incrementar el número de capas o neuronas en cada capa puede permitir al modelo capturar relaciones más complejas.
2. Aumento del tamaño de los embeddings: Aumentar la dimensión de los embeddings puede mejorar la capacidad del modelo para representar las características semánticas de las palabras.
3. Ajuste de hiperparámetros: Experimentar con diferentes tasas de aprendizaje, optimizadores y tamaños de batch.
4. Uso de pre-entrenamientos: Utilizar embeddings pre-entrenados como GloVe o Word2Vec.
5. Regularización: Agregar técnicas como Dropout o L2 regularización para evitar el sobreajuste.

# Codigo que trate de optimizar

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import numpy as np

# Semilla para reproducibilidad
seed = 1337
keras.utils.set_random_seed(seed)

# Ejemplo de datos (reemplaza esto con tus datos reales)
samples = [
    "this is a sample text",
    "another sample text for classification",
    "machine learning is fascinating",
    "deep learning models are powerful",
    "natural language processing is a complex field"
]
labels = [0, 1, 0, 1, 0]  # Ejemplo de etiquetas (reemplaza esto con tus etiquetas reales)

# Mezclar los datos
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Dividir en entrenamiento y validación
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

# Tokenización
max_tokens = 30000
max_len = 250
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=max_len)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()
y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Modelo de Transformer
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# Hiperparámetros optimizados
embed_dim = 64  # Tamaño de la incrustación
num_heads = 4  # Número de cabezas de atención
ff_dim = 128  # Tamaño de la red de alimentación
num_tokens = len(vectorizer.get_vocabulary()) + 2
maxlen = 250

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, num_tokens, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(15, activation="softmax")(x)  # Ajusta el número de clases según tu problema

modeloTransformers = keras.Model(inputs=inputs, outputs=outputs)

# Compilación del modelo
modeloTransformers.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"]
)

# Entrenamiento del modelo
history = modeloTransformers.fit(
    x_train, y_train,
    batch_size=128,
    epochs=30,
    validation_data=(x_val, y_val),
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    ]
)

# Evaluación del modelo
print(modeloTransformers.summary())

# Pruebas con el modelo entrenado
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloTransformers(x)
end_to_end_model = keras.Model(string_input, preds)

# Ejemplo de predicción
test_samples = [
    "this message is about computer graphics and 3D modeling",
    "politics and federal courts law that people understand with politician and elects congressman",
    "we are talking about religion"
]

for sample in test_samples:
    probabilities = end_to_end_model.predict([[sample]])
    print(f"Text: {sample}")
    print(f"Predicted Class: {np.argmax(probabilities[0])}")


Epoch 1/30
1/1 [==============================] - 5s 5s/step - loss: 2.0730 - accuracy: 0.5000 - val_loss: 1.3467 - val_accuracy: 1.0000
Epoch 2/30
1/1 [==============================] - 0s 133ms/step - loss: 2.0250 - accuracy: 0.5000 - val_loss: 1.2569 - val_accuracy: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 109ms/step - loss: 1.8410 - accuracy: 0.2500 - val_loss: 1.1775 - val_accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 112ms/step - loss: 1.4322 - accuracy: 0.7500 - val_loss: 1.1076 - val_accuracy: 1.0000
Epoch 5/30
1/1 [==============================] - 0s 106ms/step - loss: 1.4959 - accuracy: 0.5000 - val_loss: 1.0419 - val_accuracy: 1.0000
Epoch 6/30
1/1 [==============================] - 0s 110ms/step - loss: 1.4762 - accuracy: 0.2500 - val_loss: 0.9894 - val_accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 0s 106ms/step - loss: 1.4383 - accuracy: 0.5000 - val_loss: 0.9459 - val_accuracy: 1.0000
Epoch 8/30
1/1 [=======

## Análisis de Resultados y Reflexiones
Vamos a desglosar y analizar los resultados obtenidos tras entrenar y evaluar el modelo Transformer para clasificación de texto.

- Resultados del Entrenamiento
- Epoch 1/30

- loss: 2.0730 - accuracy: 0.5000 - val_loss: 1.3467 - val_accuracy: 1.0000
- Epoch 2/30

- loss: 2.0250 - accuracy: 0.5000 - val_loss: 1.2569 - val_accuracy: 1.0000
- Epoch 3/30

- loss: 1.8410 - accuracy: 0.2500 - val_loss: 1.1775 - val_accuracy: 1.0000
... y así sucesivamente hasta:

- Epoch 15/30

- loss: 1.0323 - accuracy: 0.2500 - val_loss: 0.8386 - val_accuracy: 1.0000
## Observaciones y Reflexiones
1. Desempeño en Entrenamiento vs Validación:

- Exactitud de Entrenamiento: La exactitud de entrenamiento fluctúa notablemente, comenzando en 50% y disminuyendo a valores tan bajos como 0% en algunas épocas. Esto sugiere que el modelo no está aprendiendo de manera consistente de los datos de entrenamiento.
- Exactitud de Validación: La exactitud de validación es consistentemente del 100% desde la primera época. Esto es un indicador claro de sobreajuste, donde el modelo ha memorizado el conjunto de validación (probablemente debido a su pequeño tamaño) y no está generalizando correctamente.
2. Pérdida de Entrenamiento y Validación:

- La pérdida de entrenamiento disminuye gradualmente, lo cual es esperado, pero la exactitud no sigue una tendencia consistente.
- La pérdida de validación también disminuye, pero dado el contexto de la exactitud de validación, esto refuerza la sospecha de sobreajuste.
Predicciones del Modelo:

3. Para los ejemplos de texto proporcionados, el modelo predice consistentemente la clase 0, lo cual indica que el modelo no está bien ajustado y probablemente está sesgado hacia una única clase, nuevamente debido al sobreajuste.


## Reflexiones
1. Tamaño del Conjunto de Datos:

- Los datos de entrenamiento y validación son extremadamente limitados. Con sólo 5 ejemplos, el modelo no tiene suficiente información para aprender adecuadamente los patrones subyacentes. En aplicaciones de NLP, se recomienda tener un conjunto de datos mucho más grande.
2. Ajuste de Hiperparámetros:

- Aunque los hiperparámetros del modelo están optimizados, con una mayor capacidad de representación, no pueden compensar la falta de datos. Debería considerarse reducir la complejidad del modelo o, idealmente, aumentar significativamente el tamaño del conjunto de datos.
3. Validación Cruzada:

- En lugar de dividir los datos en entrenamiento y validación de manera estática, usar validación cruzada puede proporcionar una mejor estimación del rendimiento del modelo, especialmente con un conjunto de datos pequeño.
4. Regularización:

- Aunque se utilizó Dropout, la regularización adicional, como el uso de una tasa de Dropout más alta, o la adición de regularización L2, puede ayudar a mitigar el sobreajuste.
5. Preprocesamiento de Datos:

- Asegurarse de que el preprocesamiento de datos sea robusto y que todas las clases estén representadas equitativamente en los datos de entrenamiento y validación.



## Propuestas para Mejorar el Modelo
1. Aumentar el Conjunto de Datos:

- Obtener más datos de entrenamiento es la prioridad número uno. Un modelo Transformer necesita una cantidad considerable de datos para entrenar efectivamente.
2. Simplificar el Modelo:

- Considerar usar un modelo menos complejo si no se pueden obtener más datos. Modelos más simples tienen menos riesgo de sobreajuste en conjuntos de datos pequeños.
3. Validación Cruzada:

- Implementar validación cruzada para obtener una mejor comprensión de la capacidad de generalización del modelo.
4. Regularización Adicional:

- Incrementar la regularización, ya sea aumentando la tasa de Dropout o añadiendo regularización L2.
5. Mejor Manejo del Preprocesamiento:

- Asegurar que el preprocesamiento de datos no introduce sesgos y que todas las clases estén equilibradas.
- En conclusión, aunque los Transformers son poderosos y versátiles, su efectividad depende en gran medida de la cantidad y calidad de los datos de entrenamiento. La clave para mejorar la precisión en este contexto es primero aumentar y mejorar el conjunto de datos, y luego ajustar el modelo en consecuencia.







### SOLUCION DE LAS 9 PREGUNTAS CON CODIGO

# Pregunta 1: Cálculo del número promedio de tokens con Spacy

In [13]:
import spacy
from sklearn.datasets import fetch_20newsgroups

# Cargar modelo de Spacy
nlp = spacy.load("en_core_web_sm")

# Descargar datos
categories = ['comp.graphics']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)

# Obtener 15 documentos
sample_texts = newsgroups_train.data[:15]

# Tokenizar y calcular promedio de tokens
num_tokens = [len(nlp(text)) for text in sample_texts]
avg_tokens = sum(num_tokens) / len(num_tokens)

print(f"Número promedio de tokens: {avg_tokens}")


Número promedio de tokens: 376.93333333333334


Explicación del Resultado de la Pregunta 1
- El resultado obtenido, Número promedio de tokens: 376.93333333333334, indica que, en promedio, los documentos de la categoría "comp.graphics" tienen aproximadamente 377 tokens. A continuación, se ofrece una explicación más detallada sobre este resultado y su relevancia en el contexto del procesamiento del lenguaje natural (NLP).

# Pregunta 2: Descarte de las primeras diez líneas

In [14]:
def read_and_clean(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        lines = file.readlines()
    cleaned_lines = lines[10:]
    return " ".join(cleaned_lines)


Las primeras diez líneas de cada fichero pueden contener metadatos, encabezados o información que no es relevante para el análisis del contenido principal del texto. Se eligen diez líneas como un número arbitrario pero razonable para asegurarse de que se excluye esta información.

In [15]:
# Función para leer y preprocesar documentos
def preprocess_text(text):
    # Dividir el texto en líneas
    lines = text.split('\n')
    # Descartar las primeras diez líneas
    lines = lines[10:]
    # Unir las líneas restantes en un solo texto
    text = '\n'.join(lines)
    return text

# Aplicar la función a todos los documentos
processed_texts = [preprocess_text(text) for text in newsgroups_train.data]


# Pregunta 3: Control del parámetro validation_split

In [16]:
validation_split = 0.2


## Pregunta 4: Ejemplo de train_samples, val_samples, train_labels, y val_labels

In [23]:
print(f"Ejemplo de train_samples: {train_samples[0]}")
print(f"Ejemplo de val_samples: {val_samples[0]}")
print(f"Ejemplo de train_labels: {train_labels[0]}")
print(f"Ejemplo de val_labels: {val_labels[0]}")


Ejemplo de train_samples: deep learning models are powerful
Ejemplo de val_samples: natural language processing is a complex field
Ejemplo de train_labels: 1
Ejemplo de val_labels: 0


# Pregunta 5: output_sequence_length

In [24]:
vectorizer = TextVectorization(max_tokens=30000, output_sequence_length=200)

In [25]:
vectorizer

# Pregunta 6: Precisión del modelo

In [ ]:
# Asumiendo que el modelo ha sido entrenado como se mostró anteriormente:
train_accuracy = model.evaluate(x_train, y_train, verbose=0)[1]
val_accuracy = model.evaluate(x_val, y_val, verbose=0)[1]

print(f"Precisión en el conjunto de entrenamiento: {train_accuracy}")
print(f"Precisión en el conjunto de validación: {val_accuracy}")


# Pregunta 7: Análisis cualitativo de la salida

In [28]:
# Ejemplos de prueba
test_samples = [
    "this message is about computer graphics and 3D modeling",
    "politics and federal courts law that people understand with politician and elects congressman",
    "we are talking about religion"
]

# Realizar predicciones y mostrar resultados
for sample in test_samples:
    probabilities = end_to_end_model.predict([[sample]])
    predicted_class = np.argmax(probabilities[0])
    print(f"Texto: {sample}")
    print(f"Clase Predicha: {predicted_class}")
    print()


1/1 [==============================] - 0s 49ms/step
Texto: this message is about computer graphics and 3D modeling
Clase Predicha: 0

1/1 [==============================] - 1s 816ms/step
Texto: politics and federal courts law that people understand with politician and elects congressman
Clase Predicha: 0

1/1 [==============================] - 0s 299ms/step
Texto: we are talking about religion
Clase Predicha: 0

